# Testing photobleaching model 

In [172]:
import numpy as np
import pandas as pd
import bokeh.io
import bokeh.plotting
import mscl.plotting
import mscl.mcmc
import scipy.signal
import matplotlib.pyplot as plt
import pymc3 as pm
import theano.tensor as tt
%matplotlib inline
bokeh.io.output_notebook()

Loading BokehJS ...

In [173]:
import imp 
imp.reload(mscl.plotting)
imp.reload(mscl.mcmc)

<module 'mscl.mcmc' from '/Users/gchure/lab/PhillipsLab201401-XXX/lab_code/mscl_analysis/mscl/mcmc.py'>

In this notebook, I will describe a heirarchical statistical model for photobleaching and test it out on simulated data. The model developed here is used to correct for signal loss due to photobleaching during the dilution experiment.  

## The problem 

The intensity $I$ of a given cell at time $t$ can be written as
$$
I(t) = \beta + I_0 \exp\left[{-t \over \tau}\right]
$$

where $\beta$ is some fixed, non-bleaching autofluorescence value, $I_0$ is the initial intensity of the cell, and $\tau$ is a characteristic decay constant with dimensions of time. 

To develop a statistical model, we can begin by writing Bayes's theorem for a single cell as 

$$
P(\beta, I_0, \tau \, \vert\, D, t) \propto P(D \, \vert \, \beta, I_0, \tau, t)P(\beta)P(I_0)P(\tau),
$$

where we have ignored the evidence $P(D)$ for simplicity. 

<br/>
### The likelihood

We can assume that for a collection of cells, the intensity at a given time $t$ should be gaussian distributed with a mean $I*$,

$$
P(D \, \vert \, \beta, I_0, \tau, t) = {1 \over \sqrt{2\pi}\sigma}\exp\left[{-(I(t) - I^*(t))^2 \over 2\sigma^2}\right]
$$


In [174]:
# Set the simulation parameters
np.random.seed(14)
n_cells = 6 
time = np.arange(0, 500, 1)
beta_opt, I0_opt, tau_opt = 1000, 8000, 100
beta_seed = np.random.normal(loc=beta_opt, scale=20, size=n_cells)
I0_seed = np.random.normal(loc=I0_opt, scale=4000, size=n_cells)
tau_seed = np.random.normal(loc=tau_opt, scale=10, size=n_cells)

def exp_decay(params, time, noise):
    beta, I0, tau = params
    return beta + I0 * np.exp(-time/tau) + noise

# Perform the simulation and store as a dataframe.
df = pd.DataFrame([], columns=['cell_number', 'I_t', 'time'])
for i in range(n_cells):
    noise = np.random.normal(loc=0, scale=450, size=len(time))
    sim = exp_decay((beta_seed[i], I0_seed[i], tau_seed[i]), time, noise)
    _df = pd.DataFrame([sim, time]).T
    _df.columns = ['I_t', 'time'] 
    _df.insert(0, 'cell_number', i) 
    df = df.append(_df, ignore_index=True)

As is typically the case, we can look at our simulated data to see if it makes sense. 

In [175]:
p = mscl.plotting.boilerplate(width=700, height=600, x_axis_label='[time]',
                      y_axis_label='intensity')

# Plot each trace individually
grouped = df.groupby('cell_number')
for g, d in grouped:
    p.line(d['time'], d['I_t'], color='slategray', alpha=0.5, legend='simulated data')
    p.circle(d['time'], d['I_t'], color='slategray', alpha=0.5) 
bokeh.io.show(p)

The data looks as is expected. Note the large degree of noise in the measurement. To get a better sense of what a single cell would look like, we can take a look at a single trace

In [176]:
# Look at just one trace
cell_ind = 3
single_cell = df.loc[df['cell_number']==cell_ind]

# Compute the "true" curve without the noise.
time_range = np.linspace(0, 500, 1000)
true_trace = beta_seed[cell_ind] + I0_seed[cell_ind] *\
                np.exp(-time_range / tau_seed[cell_ind])

# Set up the figure axis.
p = mscl.plotting.boilerplate(width=700, height=600,
                      x_axis_label='[time]', y_axis_label='intensity')

# Plot the data and the true curve.
p.circle(single_cell['time'], single_cell['I_t'], color='slategray', alpha=0.5,
        legend='simulated data')
p.line(single_cell['time'], single_cell['I_t'], color='slategray', alpha=0.5)
p.line(time_range, true_trace, color='tomato', line_width=2, legend='true bleaching curve')

# Show it in the notebook.
bokeh.io.show(p)

Since our simulated data passes the smell test, we can take a pass at inferring the relevant parameter values.  

## Approach I: Fitting to an average trajectory

There are three "reasonable" approaches to take for this experiment. The first is to compute the average trace from all cells and estimate the parameters to that average. 


In [48]:
# Compute average.
grouped = df.groupby(['time']).mean()
avg_df = grouped.reset_index()

# Plot it over all of the others.
p = mscl.plotting.boilerplate(width=800, height=700, x_axis_label='[time]',
                     y_axis_label='intensity')
p.circle(df['time'], df['I_t'], color='slategray', alpha=0.5, legend='simulated data')
p.circle(avg_df['time'], avg_df['I_t'], color='dodgerblue', legend='averaged trace')
p.line(avg_df['time'], avg_df['I_t'], color='dodgerblue')
bokeh.io.show(p)

Now perform the MCMC for the three parameters. 

In [49]:
with pm.Model() as model:
    # Define the priors. 
    beta = pm.Uniform('beta', lower=0, upper=1E6)
    I0 = pm.Uniform('I0', lower=0, upper=1E6)
    tau = mscl.mcmc.Jeffreys('tau', lower=1E-3, upper=1E6)
    
    # Evaluate the expected value.
    time = avg_df['time'].values
    mu = beta + I0 * tt.exp(-time / tau)
    
    #Define the likelihoood and sample.
    obs = avg_df['I_t'].values
    like = mscl.mcmc.MarginalizedNormal('like', mu=mu, observed=obs)
    trace = pm.sample(draws=1000, tune=5000, njobs=6)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
100%|██████████| 6000/6000 [00:12<00:00, 481.45it/s]


In [62]:
# Check convergence.
print(pm.gelman_rubin(trace))
_ = mscl.plotting.traceplot(trace, dist_type='kde')

{'beta': 1.0006815376006086, 'I0': 0.99997992605509989, 'tau': 1.0007557247208241}


As the sampler appears to be converged and the distributions don't look pathological, we can convert this trace to a data frame and compute the useful statistics. 

In [74]:
avg_mcmc_df = mscl.mcmc.trace_to_dataframe(trace, model)
stats = mscl.mcmc.compute_statistics(avg_mcmc_df)

# Display the stats.
stats['mode']
print(stats)

  parameter         mode      hpd_min     hpd_max
0      beta   993.609783   961.978004  1025.93833
1        I0  7422.598893  7353.002954  7490.07000
2       tau   104.680798   102.591854   106.82765


The modes of these distributions are right in line with what we seeded our simulation with. Let's now plot the averaged data, the "true" bleaching curve, and the results from our MCMC with the 95% credible region. 

In [91]:
# Define the time range over which we will plot.
time_range = np.linspace(0, len(time), 800)

# Compute the true curve and fit curve.
true_trace =  exp_decay((beta_opt, I0_opt, tau_opt), time_range, 0)
mcmc_trace = exp_decay(stats['mode'].values, time_range, 0)

# Compute the credible region over our time range.
cred_region = np.zeros((2, len(time_range)))
for i, t in enumerate(time_range):
    bleach = exp_decay(avg_mcmc_df.loc[:,['beta', 'I0', 'tau']].T.values, t, 0)
    cred_region[:, i] = mscl.mcmc.compute_hpd(bleach, mass_frac=0.95)

In [93]:
# Set up the figure canvas.
p = mscl.plotting.boilerplate(width=600, height=500, x_axis_label='[time]',
                     y_axis_label='signal')

# Add the data points.
p.circle(avg_df['time'], avg_df['I_t'], color='slategray', alpha=0.75, legend='averaged data')

# plot the 'true' and fit curves.
p.line(time_range, mcmc_trace, color='dodgerblue', line_width=2, legend='via MCMC')
mscl.plotting.fill_between(p, time_range, cred_region[0, :], cred_region[1, :], color='dodgerblue',
                          alpha=0.5)
p.line(time_range, true_trace, color='tomato', line_width=3, legend='"true" trace',
      line_dash='dashed')
bokeh.io.show(p)

It appears that this does a pretty great job at estimating the true parameters. While this won't look good for any individual trace, this seems like a valid approach to get a bleaching constant in a "quick and dirty" kind of way.  

##  Approach II: Fitting to individual trajectories


Another approach is to fit each trace individually and then compute a mean and standard deviation for the decay constant. 

In [96]:
# Group by cell number.
with pm.Model() as model:
    # Define the priors.
    beta = pm.Uniform('beta', lower=0, upper=1E6, shape=n_cells)
    I0 = pm.Uniform('I0', lower=0, upper=1E6, shape=n_cells)
    tau = msmc.Jeffreys('tau', lower=1E-3, upper=1E3, shape=n_cells)
    
    # Compute the expected value.
    inds = df['cell_number'].values.astype(int)
    time = df['time'].values
    theo = beta[inds] + I0[inds] * tt.exp(-time / tau[inds])
    
    # Define the likelihood.
    like = msmc.MarginalizedNormal('like', mu=theo, observed=df['I_t'].values)
    
    # Sample
    trace = pm.sample(draws=5000, tune=1000, njobs=4)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
100%|██████████| 6000/6000 [00:33<00:00, 181.01it/s]


In [21]:
# Congeal the dataframe and append
indiv_df = pm.trace_to_dataframe(trace)
indiv_df['logp'] = pm.stats._log_post_trace(trace, model).sum(axis=1)
ind = np.argmax(indiv_df['logp'])
modes = indiv_df.iloc[ind]

Plot each fit against each individual trace

In [24]:
canvases = [[], []]
grouped = df.groupby('cell_number')
_modes = modes.to_dict()
for g, d in grouped:
    # Compute the true trace and fit trace.
    true_trace = exp_decay((beta_seed[g], I0_seed[g], tau_seed[g]), time_range, 0)
    mcmc_trace = exp_decay(modes[['beta__{0}'.format(g),'I0__{0}'.format(g),
                                   'tau__{0}'.format(g)]], time_range, 0)
   
    # Set up the figure
    p = mscl.bokeh_boiler(width=300, height=300, title='cell {0}'.format(g))
    p.circle(d['time'], d['I_t'], color='slategray', alpha=0.5) 
    p.line(time_range, true_trace, line_width=3, color='tomato')
    p.line(time_range, mcmc_trace, line_width=2, color='dodgerblue')
    
    if g < 3:
        canvases[0].append(p)
    else:
        p.xaxis.axis_label = '[time]'
        canvases[1].append(p)
        
# Show the grid plot
layout = bokeh.layouts.gridplot(canvases)
bokeh.io.show(layout)

Of course, this represents each trace individually well. 

## Approach III: A heirarchical model

In [163]:
# Make a new dataframe with some smoothed data included.
with pm.Model() as model:
    # Set the top level hyperpriors.
    tau_mu = pm.Uniform('tau_mu', lower=1E-3, upper=1E3)
    beta_mu = pm.Uniform('beta_mu', lower=0, upper=1E6)
    tau_sig = msmc.Jeffreys('tau_sig', lower=1E-2, upper=1E3)
    beta_sig = msmc.Jeffreys('beta_sig', lower=1E-2, upper=1E3)

    # Set the lower level priors -- Reparameterized.
    tau = mscl.mcmc.ReparameterizedNormal('tau', mu=tau_mu, sd=tau_sig,
                                          shape=n_cells)
    beta = mscl.mcmc.ReparameterizedNormal('beta', mu=beta_mu, sd=beta_sig, 
                                           shape=n_cells)  
    I0 = pm.Uniform('I0', lower=0, upper=1E6, shape=n_cells)
    
    # Get the ID's and other observables
    ids = df['cell_number'].values.astype(int)
    time = df['time'].values.astype(float)
    obs = df['I_t'].values.astype(float)
    
    # Compute the expected value.
    theo = beta[ids] + I0[ids] * tt.exp(-time / tau[ids])
    
    # Set the likelihood.
    like = msmc.MarginalizedNormal('like', mu=theo, observed=obs)  
    trace = pm.sample(draws=5000, tune=5000, njobs=4)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
 95%|█████████▍| 9475/10000 [03:42<00:12, 42.50it/s] /Users/gchure/anaconda/lib/python3.6/site-packages/pymc3/step_methods/hmc/nuts.py:451: UserWarning: The acceptance probability in chain 3 does not match the target. It is 0.667211132225, but should be close to 0.8. Try to increase the number of tuning steps.
  % (self._chain_id, mean_accept, target_accept))
/Users/gchure/anaconda/lib/python3.6/site-packages/pymc3/step_methods/hmc/nuts.py:467: UserWarning: Chain 3 contains 159 diverging samples after tuning. If increasing `target_accept` does not help try to reparameterize.
  % (self._chain_id, n_diverging))
 96%|█████████▌| 9622/10000 [03:46<00:08, 42.57it/s]/Users/gchure/anaconda/lib/python3.6/site-packages/pymc3/step_methods/hmc/nuts.py:467: UserWarning: Chain 2 contains 27 diverging samples after tuning. If increasing `target_accept` does not help try to reparameterize.
  % (self._chain_id, n_diverging))
1

In [168]:
# As usual, we can look at the trace to see how how well things were behaved
_ = mscl.plotting.traceplot(trace, varnames=['tau_mu', 'tau_sig', 'beta_mu', 'beta_sig'], dist_type='kde')

It looks like we have one diverging chain, but the rest look like they've reached convergence. We can now compute the stats, and credible regions, and plot the results.

In [169]:
# Compute the statistics.
heirarchical_df = mscl.mcmc.trace_to_dataframe(trace, model)
heirarchical_stats = mscl.mcmc.compute_statistics(heirarchical_df)

To see how well we did, we can plot the MCMC result for each individual cell and see how it compares to the "true" curve. 

In [170]:
canvases = [[], []]
grouped = df.groupby('cell_number')

# Compute the mode for beta and tau
tau_mode, beta_mode = heirarchical_stats.loc[(heirarchical_stats['parameter']=='beta_mu') | 
                       (heirarchical_stats['parameter']=='tau_mu'), 'mode'].values
for g, d in grouped:
    # Compute the true trace and fit trace.
    true_trace = exp_decay((beta_seed[g], I0_seed[g], tau_seed[g]), time_range, 0)
    params = (beta_mode, heirarchical_stats.loc[heirarchical_stats['parameter']=='I0__{0}'.format(g), 'mode'].values,
              tau_mode) 
    mcmc_trace = exp_decay(params, time_range, 0)
    
    # Compute the credible region.
    cred_region = np.zeros((2, len(time_range)))
    for i, t in enumerate(time_range):
        params = heirarchical_df.loc[:, ['beta_mu', 'I0__{0}'.format(g), 'tau_mu']].T.values
        trace = exp_decay(params, t, 0)
        cred_region[:, i] = mscl.mcmc.compute_hpd(trace, mass_frac=0.95)
        
                            
    # Set up the figure
    p = mscl.plotting.boilerplate(width=300, height=300, title='cell {0}'.format(g),
                                 y_axis_label='signal', x_axis_label='[time]')
    
    # Plot the data
    p.circle(d['time'], d['I_t'], color='slategray', alpha=0.3) 

    # Plot the MCMC result + credible region
    p.line(time_range, mcmc_trace, line_width=2, color='dodgerblue')
    mscl.plotting.fill_between(p, time_range, cred_region[0,:], cred_region[1, :],
                              color='dodgerblue', alpha=0.5)
    
    # Plot the true decay. 
    p.line(time_range, true_trace, line_width=3, color='tomato', line_dash='dashed')   
    
    # Determine where to plot the figure.
    if g < 3:
        canvases[0].append(p)
    else:
        p.xaxis.axis_label = '[time]'
        canvases[1].append(p)
        
# Show the grid plot
layout = bokeh.layouts.gridplot(canvases)
bokeh.io.show(layout)

By zooming into the plot above, we cans ee that the true bleaching curve and the MCMC estimate are very close. Where they are not overlapping, the two are within the credible region of our estimate.

## So which is better? 

In short, basically every approach in this notebook yielded about the same result. However, I believe that the heirarchical model is the most philosophically satisfying and will use that to determine the actual bleaching constant for these experiments.